### Collecting some ping statistics

Ping data collected every 5 seconds doing single pings to 8.8.8.8

**Example**:
>$ ping -c 1 8.8.8.8  
>PING 8.8.8.8 (8.8.8.8): 56 data bytes  
>64 bytes from 8.8.8.8: icmp_seq=0 ttl=119 **time=22.985 ms**  
>--- 8.8.8.8 ping statistics ---  
>1 packets transmitted, 1 packets received, **0.0% packet loss**  
>round-trip min/**avg**/max/**stddev** = 22.985/22.985/**22.985**/**0.000** ms  

Data beeing collected:
 - **Ping latency** (in miliseconds, 22.985 ms in the example)
 - __Percentage of packets dropped__  (0.0% in the example, can be only 0% or 100%, data collected for single pings) 
 - **Standart deviation (for latency)**  - always 0 for single pings


In [1]:
from influxdb import DataFrameClient
from influxdb import InfluxDBClient

from plotly.offline import init_notebook_mode, iplot
import plotly.graph_objs as go
import plotly.plotly as py
from plotly import tools
init_notebook_mode(connected=True)

Set up influxdb connection:

In [2]:
host='204.209.76.145'
port=8086
dbname = 'net_speed_md'
client = InfluxDBClient(host, port, '', '', dbname)

 Used this to learn about influxdb:
 - https://influxdb-python.readthedocs.io/en/latest/examples.html
 - https://www.influxdata.com/blog/getting-started-python-influxdb/
 - https://influxdb-python.readthedocs.io/en/latest/resultset.html
 - https://influxdb-python.readthedocs.io/en/latest/api-documentation.html
 - https://docs.influxdata.com/influxdb/v1.0//query_language/functions  - Functions!!

In [4]:
#Trying to load entire table, convert to dataframe and estimate how much memory its taking
#from here https://stackoverflow.com/questions/18089667/how-to-estimate-how-much-memory-a-pandas-dataframe-will-need
#client = DataFrameClient(host, port, '', '', dbname)
#query_ping = 'SELECT * FROM PING;'
#result_ping = client.query(query_ping)
#ping_df = result_ping['PING']
#print(ping_df.memory_usage())
#print(sys.getsizeof(ping_df))
#rint( df.memory_usage(deep=True).sum())

Checking last 20 records in the ping measurment and see how data looks like:

In [5]:
client_df = DataFrameClient(host, port, '', '', dbname)
query_ping = 'SELECT PING,SK_PI FROM PING ORDER BY time DESC LIMIT 10;'
result_ping = client_df.query(query_ping)
ping_df = result_ping['PING']
ping_df

,PING,SK_PI
2018-12-19 23:59:57.996999936+00:00,0.000,3
2018-12-19 23:59:58.996999936+00:00,43.154,18
2018-12-19 23:59:58.996999936+00:00,32.492,9
2018-12-19 23:59:58.996999936+00:00,35.068,15
2018-12-19 23:59:58.996999936+00:00,0.000,16
2018-12-20 00:00:00+00:00,27.172,11
2018-12-20 00:00:00+00:00,0.000,10
2018-12-20 00:00:00+00:00,0.000,17
2018-12-20 00:00:00+00:00,0.000,3
2018-12-20 00:00:00+00:00,80.602,12


Let's take just one device, for example 3:

In [6]:
query_ping = "SELECT PING FROM PING WHERE SK_PI='3' ORDER BY time DESC LIMIT 10;"
result_ping = client_df.query(query_ping)
ping_df = result_ping['PING']
ping_df

,PING
2018-12-19 23:59:41.980000+00:00,14.701
2018-12-19 23:59:43.983000064+00:00,0.000
2018-12-19 23:59:45.987000064+00:00,14.610
2018-12-19 23:59:47.987000064+00:00,0.000
2018-12-19 23:59:49.990000128+00:00,0.000
2018-12-19 23:59:51.990000128+00:00,14.740
2018-12-19 23:59:53.992999936+00:00,0.000
2018-12-19 23:59:55.996999936+00:00,14.666
2018-12-19 23:59:57.996999936+00:00,0.000
2018-12-20 00:00:00+00:00,0.000


Lets compare with what we have in mysql database.

In [8]:
import pyodbc
import pandas as pd
password='' #from 1Password! 
cnxn = pyodbc.connect(driver='/usr/local/lib/libtdsodbc.so', server='198.62.164.54',port='1433', database='net_speed_md', uid='cybera_sql', pwd=password)
sql = "SELECT TOP 10 DATA_DATE, PING FROM FCT_PI where sk_pi='3' ORDER BY DATA_DATE DESC;"
data = pd.read_sql(sql,cnxn)
data

,DATA_DATE,PING
0,2018-12-20 23:59:59.400,NaN
1,2018-12-20 23:59:57.397,14.707
2,2018-12-20 23:59:55.397,NaN
3,2018-12-20 23:59:53.393,NaN
4,2018-12-20 23:59:51.390,14.699
5,2018-12-20 23:59:49.390,NaN
6,2018-12-20 23:59:47.387,14.696
7,2018-12-20 23:59:45.387,NaN
8,2018-12-20 23:59:43.383,NaN
9,2018-12-20 23:59:41.380,14.563


Pings are coming every 5 seconds. Missing values are NaN in original and 0 in the influxdb (we might consider re-importing and omitting values with NaN)

Checking statistics for number of packets dropped:

In [9]:
query_ping = "SELECT DISTINCT(PING_DROPRATE) from PING;"
result_ping = client_df.query(query_ping)
ping_df = result_ping['PING']
ping_df

,distinct
1970-01-01 00:00:00+00:00,0
1970-01-01 00:00:00+00:00,100


In Grafana we see that the device #10 has  some packets dropped, lets see what is the value of *PING* measurment when *PING_DROPRATE* is equal to 100:

In [11]:
query_ping = "SELECT PING, PING_DROPRATE FROM PING WHERE SK_PI='10' ORDER BY time DESC LIMIT 10;"
result_ping = client_df.query(query_ping)
ping_df = result_ping['PING']
ping_df

,PING,PING_DROPRATE
2018-12-19 23:59:41.980000+00:00,45.827,0
2018-12-19 23:59:43.983000064+00:00,0.000,0
2018-12-19 23:59:45.987000064+00:00,0.000,0
2018-12-19 23:59:47.987000064+00:00,45.482,0
2018-12-19 23:59:49.990000128+00:00,0.000,0
2018-12-19 23:59:51.990000128+00:00,46.267,0
2018-12-19 23:59:53.992999936+00:00,0.000,0
2018-12-19 23:59:55.996999936+00:00,0.000,0
2018-12-19 23:59:57.996999936+00:00,0.000,100
2018-12-20 00:00:00+00:00,0.000,0


*PING* is equal to 0 in this case, its importatnt to know.

### Number of datapoints per device

Getting device numbers(tags SK_PI):

In [12]:
query_unique_devices = "SHOW TAG VALUES FROM PING WITH KEY=SK_PI;"
result_unique_devices = client.query(query_unique_devices)
points_unique_devices = result_unique_devices.get_points()
device_numbers=[]
for point in points_unique_devices:
    device_numbers.append(point['value'])
device_numbers=list(map(int, device_numbers))
device_numbers= sorted(device_numbers)
print(device_numbers)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


Getting number of data points per device for the entire period of time.

In [13]:
query_ping_counts = 'SELECT COUNT(PING) FROM PING GROUP BY SK_PI;'
result_ping_counts = client.query(query_ping_counts)

In [14]:
device_counts=[]
for device in device_numbers:
    points_ping_counts=result_ping_counts.get_points(tags={'SK_PI':str(device)})
    for point in points_ping_counts:
        device_counts.append(point['count'])

Plotting device numbers and number of data points. 

In [15]:
data = [go.Bar(
            x=device_numbers,
            y=device_counts
    )]

layout = go.Layout(
        barmode='stack',
        title="Number of data points per device"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Some of the devices have small number of datapoints, may be they are just installed? Lets check how many dataponts came in December.

Getting number of datapoints per device in December and list of the devices that have data for December(not all of the have).

In [22]:
query_ping_counts_dec = "SELECT COUNT(PING) FROM PING WHERE time >= '2018-12-01 00:00:00' GROUP BY SK_PI ;"
# another option - check last 4 weeks
#query_ping_counts_dec = "SELECT COUNT(PING) FROM PING WHERE time >= now()-4w GROUP BY SK_PI ;"
result_ping_counts_dec = client.query(query_ping_counts_dec)

In [23]:
device_counts_dec=[]
device_numbers_dec = []
for device in device_numbers:
    points_ping_counts_dec=result_ping_counts_dec.get_points(tags={'SK_PI':str(device)})
    point_new=0
    for point in points_ping_counts_dec:
        point_new=point['count']
    if (point_new!=0): 
        device_numbers_dec.append(device)
    device_counts_dec.append(point_new)
print("Devices, that have data in December: ",device_numbers_dec)

Devices, that have data in December:  [3, 4, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


Plotting combined barchart - entire number of datapoints vs number of datapoints in December.

In [24]:
trace1 = go.Bar(
            x=device_numbers,
            y=device_counts_dec,
            name='December',
    )
trace2 = go.Bar(
            x=device_numbers,
            y=[a - b for a, b in zip(device_counts, device_counts_dec)],
            name='Entire time',
    
    )
data = [trace1, trace2]
layout = go.Layout(
        barmode='stack',
        title="Comparing number of datapoints in December vs entire time"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Devices 1,2,4,5,6,7 and possibly 13 and 14 need to be double checked. Looks like they have started reporting and then stopped. Let's check last reporting time for every device.

In [25]:
query_ping_last = "SELECT last(PING), time FROM PING GROUP BY SK_PI;"
result_ping_last = client.query(query_ping_last)

In [26]:
for device in device_numbers:
    points_ping_last=result_ping_last.get_points(tags={'SK_PI':str(device)})
    for point in points_ping_last:
        print("Device: ", device," reported last time on ", point['time'])

Device:  1  reported last time on  2018-10-11T15:46:31.790000128Z
Device:  2  reported last time on  2018-11-06T19:16:32.792999936Z
Device:  3  reported last time on  2018-12-20T00:00:00Z
Device:  4  reported last time on  2018-12-05T22:53:02.383000064Z
Device:  5  reported last time on  2018-11-05T21:32:32.552999936Z
Device:  6  reported last time on  2018-10-31T14:55:02.703000064Z
Device:  7  reported last time on  2018-12-18T08:46:02.763000064Z
Device:  8  reported last time on  2018-12-03T19:58:50.129999872Z
Device:  9  reported last time on  2018-12-19T23:59:58.996999936Z
Device:  10  reported last time on  2018-12-20T00:00:00Z
Device:  11  reported last time on  2018-12-20T00:00:00Z
Device:  12  reported last time on  2018-12-20T00:00:00Z
Device:  13  reported last time on  2018-12-04T20:27:07.227000064Z
Device:  14  reported last time on  2018-12-06T02:12:31.350000128Z
Device:  15  reported last time on  2018-12-19T23:59:58.996999936Z
Device:  16  reported last time on  2018-12-

Something is happening with devices  1,2,4,5,6,8,13 and 14. They stopped reporting - does it need to be investigated?

### Ping latency 

Let's check actual ping latency numbers for December:

In [37]:
query_ping_max_dec = "SELECT MAX(PING), MEAN(PING) FROM PING WHERE time >= '2018-12-01 00:00:00' GROUP BY SK_PI;"
result_ping_max_dec = client.query(query_ping_max_dec)

We will exclude devices  1,2,4,5,6,8,13,14 as they are not reporting any more

In [38]:
#device_numbers_dec = [x for x in device_numbers if x not in [1,2,4,5,6,8,13,14]]
#print(device_numbers_dec)

In [39]:
device_max_dec=[]
device_mean_dec = []
for device in device_numbers_dec:
    points_max_dec=result_ping_max_dec.get_points(tags={'SK_PI':str(device)})
    for point in points_max_dec:
        device_max_dec.append(point['max'])
        device_mean_dec.append(point['mean'])

Plotting mean and max ping latency results for every reporting result in December.

In [40]:
trace1 = go.Bar(
            x=device_numbers_dec,
            y=device_mean_dec,
            name='Mean',
    )
trace2 = go.Bar(
            x=device_numbers_dec,
            y=device_max_dec,
            name='Max',
    
    )
data = [trace1, trace2]
layout = go.Layout(
       # barmode='stack',
        title="Maximum and mean delay in miliseconds"
    )

fig = go.Figure(data=data, layout=layout)
iplot(fig)

Device 16 has the highest mean, it needs to be, (all these spikes are also visible in Grafana). Something is going on there. Device 13 looks like it was giving high latency but stopped reporting Dec 4th

### To do: 
Find median?  (Tried to use median function from influxdb but it always returns 0, too many zeros in the database?)  
Check by time of the day?  
Divide ping latencies into groups  and calculate percentage of devices by group?  

### Number of packets dropped 
We will calculate the etite number of packets dropped by device using formula:SUM(PING_DROPRATE)/100

In [48]:
query_dropped_counts = "SELECT SUM(PING_DROPRATE)/100 FROM PING GROUP BY SK_PI ;"
result_dropped_counts = client.query(query_dropped_counts)

In [49]:
device_dropped_counts=[]
for device in device_numbers:
    points_dropped_counts=result_dropped_counts.get_points(tags={'SK_PI':str(device)})
    for point in points_dropped_counts:
        device_dropped_counts.append(point['sum'])
print(device_dropped_counts)
print(sum(device_dropped_counts))

[36, 9881, 221, 82788, 1520, 2484, 1502, 15285, 3280, 236190, 5002, 2520, 269, 474, 1319, 9992, 578, 8996]
382337


In [50]:
trace = go.Pie(labels=device_numbers, values=device_dropped_counts)
data=[trace]
layout = go.Layout(
        title="Total number of packets dropped by device"
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

Device #10 and device #7  have the largest number of packets dropped.
We will do the same for December:

In [53]:
query_dropped_counts_dec = "SELECT SUM(PING_DROPRATE)/100 FROM PING WHERE time >= '2018-12-01 00:00:00' GROUP BY SK_PI ;"
result_dropped_counts_dec = client.query(query_dropped_counts_dec)

In [54]:
device_dropped_counts_dec=[]
for device in device_numbers:
    points_dropped_counts_dec=result_dropped_counts_dec.get_points(tags={'SK_PI':str(device)})
    point_sum=0
    for point in points_dropped_counts_dec:
        point_sum=point['sum']
    device_dropped_counts_dec.append(point_sum)    

In [55]:
trace = go.Pie(labels=device_numbers, values=device_dropped_counts_dec)
data=[trace]
layout = go.Layout(
        title="Total number of packets dropped by device in December"
    )
fig = go.Figure(data=data, layout=layout)
iplot(fig)

We still have issues with device #10  as well as 4 and 16.

### To do: 
- By time of the day?
- Relatively to the number of datapoints(pings)?